1. 문서의 내용을 읽기
2. 문서를 쪼개기
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 쪼갠 문서를 임베딩해서 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 llm에 질문과 함께 전달 

### 문서 내용 읽기

In [1]:
%pip install --upgrade --quiet docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter # 다양한 TextSplitter들중 하나

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 하나의 청크에 가지는 토큰수
    chunk_overlap=200, #  앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정
                       # 가지고 올때 전에 가져온거의 멏줄 정도 겹쳐서 가져옴.(앞뒤 문맥 파악을 위해)
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [32]:
len(document_list)

187

### 임베딩

In [20]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_openai import OpenAIEmbeddings


# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
# https://platform.openai.com/docs/pricing

In [ ]:
%pip install langchain-chroma

In [28]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

### 답변 생성을 위한 Retrieval

Chroma에 저장한 데이터를 유사도 검색(similarity_search())를 활용해서 가져옴

In [33]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
# 존댓말 쓰면 답변을 좀 더 잘해줌.

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k = 3)

In [11]:
retrieved_docs

[Document(id='c0930651-27d9-474c-a165-58a0acd53bb2', metadata={'source': './tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금

### Augmentation을 위한 Prompt 활용

Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')
# https://platform.openai.com/docs/pricing

In [13]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.format

[Context]
{retrieved_docs}

Question: {query}
"""

In [14]:
ai_message = llm.invoke(prompt)

ai_message.content

'직장인의 소득세를 계산하기 위해서는 소득세의 세율과 기본 공제를 고려해야 합니다. 2023년 기준으로 한국의 소득세는 다음과 같은 구간별 세율이 적용됩니다:\n\n1. 1,200만원 이하: 6%\n2. 1,200만원 초과 ~ 4,600만원 이하: 15%\n3. 4,600만원 초과 ~ 8,800만원 이하: 24%\n\n연봉 5천만원인 직장인의 과세소득은 다음과 같이 계산됩니다.\n\n1. 기본 공제: 150만원 (근로소득자의 경우)\n2. 총소득: 5,000만원\n3. 과세 소득 = 총소득 - 기본 공제 = 5,000만원 - 150만원 = 4,985만원\n\n이제 과세 소득에 대해 세율을 적용하여 소득세를 계산합니다.\n\n1. 1,200만원 이하: 1,200만원 * 6% = 72만원\n2. 1,200만원 초과 ~ 4,600만원: (4,600만원 - 1,200만원) * 15% = 51만원\n3. 4,600만원 초과 ~ 4,985만원: (4,985만원 - 4,600만원) * 24% = 92.4만원\n\n총 소득세 = 72만원 + 51만원 + 92.4만원 = 215.4만원\n\n따라서, 연봉 5천만원인 직장인의 대략적인 소득세는 약 215.4만원입니다. 다만, 최종 세액은 세액공제, 추가 공제 등을 적용하여 조금 달라질 수 있습니다. 정확한 계산을 원하신다면 세무 전문가와 상담하시는 것이 좋습니다.'

In [27]:
# 다 했지만 
# 랭'체인'인 만큼
# 리트리버를 좀 더 효과적으로 해보자.
# RetrievalQA Chain을 써보자 (Retrieval Question Aanswer)

In [18]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain import hub
# langchain hub는 많은 기능들중 가장 큰 것은
# 효과적인 프롬프트를 준다.
prompt = hub.pull("rlm/rag-prompt")

c:\Users\BaekSeungJin\rag-practice\.venv\Lib\site-packages\langsmith\client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [22]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### 답변 생성

In [34]:
from langchain.chains import RetrievalQA

"""
기존에는 
데이터베이스를 생성하고,          # database = Chroma(collection_name='chroma-tax'~~)
유사도검색을 한 docs를 가져오고,  # retrieved_docs = database.similarity_search(query, k = 3)
프롬프트를 직접 짜고,             # prompt = ~~~
llm에 인보크 했었다.              # ai_message = llm.invoke(prompt)
"""

"""
하지만 RetrievalQA를 사용하여 매개변수에 넣어주면
뒷단에 알아서 데이터가 들어가게 되고,
우리는 query만 해주면 된다.
"""
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [35]:
ai_message = qa_chain({"query": query})

In [ ]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '죄송하지만, 주어진 맥락에 연봉 5천만원에 해당하는 소득세 계산에 대한 정보가 없습니다. 소득세는 일반적으로 소득 수준과 세액 공제 등을 기반으로 하여 계산되며, 이에 대한 구체적인 세율이나 공제 항목이 필요합니다. 정확한 소득세를 알고 싶다면 구체적인 세율이나 산출 기준을 확인해야 합니다.'}